<a href="https://colab.research.google.com/github/SKMAAX/BokehCsvGraphViewer/blob/main/BokehCsvGraphViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Python BokehでCSV描画用htmlを生成する [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SKMAAX/BokehCsvGraphViewer/blob/main/BokehCsvGraphViewer.ipynb)

In [ ]:
from bokeh.layouts import row, column
from bokeh.models import Div, ColumnDataSource, CustomJS, FileInput, RangeTool, Range1d, HoverTool
from bokeh.plotting import figure, save, output_file
from bokeh.palettes import Category10

output_file("CsvGraphViewer.html", mode='inline') # オフラインで使うためinline設定
ColumnList = ["x","y1","y2","y3","y4","y5","y6"] # 列ラベル(TODO:これもCSVファイルの列ラベルから更新できるはず…)
initlist = [[0] for i in range(len(ColumnList))] # [[0],[0],・・・,[0]]

dict_data = dict(zip(ColumnList,initlist))
source = ColumnDataSource(data=dict_data)

colors = list(Category10.values())[5] # 適当なカラーパレットを選択

# マウスオーバーしたときに表示される項目の設定
tooltips = [
    ("TimeCnt", "@x"),
    ("(y1,y2)", "(@y1,@y2)"),
    ("(y3,y4)", "(@y3,@y4)"),
    ("(y5,y6)", "(@y5,@y6)"),
]

# 描画領域の設定
plot = figure(
    plot_width=400, 
    plot_height=400,
    x_range = Range1d(0,4096), # 横軸データ点数4096点 (TODO:これもCSVファイルの内容から更新できるはず…)
    tooltips=tooltips,
    )

# グラフの描画
for i in range(len(ColumnList)-1):
    plot.line('x', list(dict_data.keys())[i+1], source=source, line_width=2, line_alpha=0.6, legend_label=list(dict_data.keys())[i+1], color=colors[i])
    #plot.circle('x', list(dict_data.keys())[i+1], source=source, color=colors[i]) #ドット表示する場合

plot.left[0].formatter.use_scientific = False # 指数表記にしない
plot.legend.click_policy="hide" # 凡例をクリックするとグラフ表示/非表示
#plot.add_tools(HoverTool()) # マウスオーバーしたときに、詳細をポップアップ表示 → tooltipsで与えているので、不要

# RangeTool(横方向の拡大縮小ができる)の設定
range_plot = figure(
    plot_height=200,
    plot_width=plot.width,
    y_range=plot.y_range,
    toolbar_location=None,
  )

range_plot.line('x', 'y3', source=source) # RangeToolにはy3を表示させる
range_rool = RangeTool(x_range=plot.x_range)
range_plot.add_tools(range_rool)

fi_label = Div(text='CsvGraphViewer') # divタグウィジェット
fi = FileInput() # ファイル選択ウィジェット

# ファイル選択時に実行されるコールバック関数の記述
callback = CustomJS(
    args=dict(source=source), # コールバック関数に渡すデータソース
    code=""" // コールバック関数の記述(JavaScript)
    Papa.parse(atob(cb_obj.value), {
        delimiter: ',',
        header: true,
        dynamicTyping: true,
        worker: true,
        complete: function (results) {

            // csvデータを格納するアキュムレータの初期化
            const acc = results.meta.fields.reduce((acc, f) => {
                acc[f] = [];
                return acc;
            }, {}); 

            // csvを1行データ(row)ごとに読み取り、さらに列(k)ごとに読み取ってaccに積み上げていく
            // 最終行まで終わったら、積み終わったaccを更新後データとしてsorce.dataに渡す。
            source.data = results.data.reduce((acc,row,index) => {
                for (const k in acc) {
                    acc[k].push(row[k]);
                    }
                return acc;
            }, acc); 

        }
    });
""")

# ファイル選択の内容が変わったらcallbackが実行されるようにセット
# (同一ファイルを再選択してもcallbackは起動しない)
fi.js_on_change('value', callback)

# JavaScript内で使っているCSVパーサPapaParse用のテンプレート。ブラウザで起動時最初にここにアクセスしに行く。
# papaparse.min.jsを落としてくれば、完全ローカル化もできる。
template = """\
{% block preamble %}
<script src="https://cdnjs.cloudflare.com/ajax/libs/PapaParse/5.1.0/papaparse.min.js"
        integrity="sha256-Fh801SO9gqegfUdkDxyzXzIUPWzO/Vatqj8uN+5xcL4="
        crossorigin="anonymous"></script>
{% endblock %}
"""

# 各ウィジェットの配置を設定
layout = column(row(fi_label, fi), plot, range_plot)

# PapaParseのテンプレートを使う関係上、show()でのスクリプト実行時のブラウザ起動はできない。
# 本スクリプトではhtmlファイルの保存のみを行う。
save(layout, template=template)

'/content/CsvGraphViewer.html'